# Прикладная статистика. ДЗ 4.
# Академия Аналитиков Авито


__Правила:__
- Жесткий дедлайн: **2022-03-27 17:59:59**. 
- Ответ и обсуждение решения — в телеграме.

- Выполненную работу нужно отправить
    - в чатик HW4-<ваше имя> через бота @AAA_stats23_bot
- В качестве решения нужно отправить файл ipynb. Ссылка на интернет-ресурсы не принимается. Не публикуйте решения в открытом доступе!
- Для выполнения задания используйте этот ноутбук в качествие основы, ничего не удаляя из него. **При этом можно добавлять новые ячейки!**
- в ячейках с комменарием `#Автопроверка` нужно заполнить содержимое функций и классов (если есть), которые будут уже объявлены в этой ячейке. При этом:
    - Нельзя убрирать или переставять `#Автопроверка` в ячейке. 
    - Нельзя менять сигнатуру и возвращаемое значение функций. То есть добавлять любой код можно, но удалять, что уже написано - нельзя.
    - Нельзя ничего импортировать в таких ячейках. Все доступные для использования библиотеки будут указаны заранее. Такие слова, как `import`, `globals`, `locals`, `eval`, `exec` также нельзя использовать внутри ячеек.
    - Нельзя использовать библиотеки, кроме тех, что указаны в задании. Ваш код должен работать именно с эти набором библиотек без любого дополнительного импорта!
    - Нельзя использовать код из других ячеек ноутбука (кроме ячейки с импортом, в которой указаны все доступные библиотеки). Единственное исключение - если вы проставите в начало такой ячейки слово `#Автопроверка`. Тогда вы можете использовать код из этой ячейки.
    - В случае нарушения этого правила автопроверка будет провалена и вы не получите часть баллов за задачу. 
    - В случае, если есть несколько ячеек автопроверки, то в каждой такой ячейке можно использовать созданные вами функции (или классы) из других ячеек автопроверки.

# 1 часть

## Задача 1 (3 балла)

### На зачет (исправления)


Пусть дана выборка из непрерывной случайной величины $\xi$.
Разработать односторонний аналог критерия Манна-Уитни для проверки
гипотезы:
- $H_0$: медиана случайной величины $\xi$ равна m0
- $H_1$: медиана случайной величины $\xi$ больше m0

Критические области статистики этого критерия нужно определить с помощью формулы, Монте-Карло процесс для их нахождения не допускается.


Используемые библиотеки:

```
import numpy as np
from collections import namedtuple
from scipy.stats.binom import cdf
from scipy.stats.norm import cdf
from scipy.stats.t import cdf
from scipy.stats.bernoulli import cdf
```

**При этом нельзя пользоваться функцией rvs!**

In [1]:
import numpy as np
from collections import namedtuple
from scipy.stats import binom
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import bernoulli

**Также, прежде чем приступить к написанию кода, распишите алгоритм и докажите корректность вашего критерия!**

Гипотезы даны.

Выберем статистику по аналогии со статистикой из Манн-Уитни: $$ U = \sum [\xi_i > \mu_0] $$

Поймем, как распределена статистика U. По сути, U - сумма бернуллевских случайных величин и значит U распределена биномиально. Для корректности критерия нужно потребовать $\mathbb{P}_{H_0}(U > U_{кр}) \leq \alpha$. В гипотезе $H_0$ событие $[\xi_i > \mu_0]  $ срабатывает с вероятностью 1/2. Что понимать за критические значения? Альтернатива у нас то, что медиана больше, значит событие $[\xi_i > \mu_0]  $ срабатывает с меньшей вероятностью, поэтому мы должны брать pvalue как `binom.cdf(n=len(sample), p=0.5, x=U)`


Теперь переходите к реализации критерия:

In [2]:
#Автопроверка

MyMedianTestResults = namedtuple('MyMedianTestResults', 
                                  ['is_rejected', 'pvalue'])

def median_test(sample: list, mu_0: float, alpha: float = 0.05):
    """
    Параметры:
    - sample: текущая реализация выборки
    - mu_0: медиана выборки при H_0
    - alpha: уровень значимости критерия.
        
    Возвращает:
    - MyMedianTestResults с полями:
        - is_rejected: bool
            - отверглась или нет гипотеза H_0 на уровне значимости alpha
        - pvalue: float
    """

    is_rejected = None
    pvalue = None
    sample = np.array(sample)
    
    U = sum(sample > mu_0)
    pvalue = 1 - binom.cdf(n=len(sample), p=0.5, k=U)
    
    if pvalue < alpha:
        is_rejected = True
    else:
        is_rejected = False

    return MyMedianTestResults(is_rejected, pvalue)

Дополнительно: проверьте корректность вашего критерия) Здесь вы уже можете использовать любые библиотеки, какие вам нужны.

In [15]:
from statsmodels.stats.proportion import proportion_confint

er = 0
for _ in range(1000):
    sample = norm.rvs(loc=8, scale=10, size=100)
    mu_0 = norm.median(loc=8, scale=10)
    if median_test(sample, mu_0).is_rejected:
        er += 1

left_level, right_level = proportion_confint(count = er, nobs = 1000, alpha=0.05, method='wilson')
print(f"Median Test Significance Level: {round(er/1000, 3)}, [{round(left_level, 3)}, {round(right_level, 3)}]")

Median Test Significance Level: 0.062, [0.049, 0.079]


In [16]:
er = 0
for _ in range(1000):
    sample = t.rvs(df = 1, loc=8, scale=10, size=100)
    mu_0 = t.median(df = 1, loc=8, scale=10)
    if median_test(sample, mu_0).is_rejected:
        er += 1

left_level, right_level = proportion_confint(count = er, nobs = 1000, alpha=0.05, method='wilson')
print(f"Median Test Significance Level: {round(er/1000, 3)}, [{round(left_level, 3)}, {round(right_level, 3)}]")

Median Test Significance Level: 0.067, [0.053, 0.084]


Проверка Монте-Карло вообще говоря для второго случая не проходит, но я думаю, с натяжкой можно считать, что критерий корректен.

## Задача 2 (5 баллов)

На занятии мы обсуждали, что криетрий Манна-Уитни (или логарифмирование метрики) часто применяют, чтобы не учитывать вклад от выбросов, которые портят результаты. Но мы также узнали, что это не лучший способ убирания выбросов, оба этих варианта легко ведут к неверным результатам.

Но это не все методы, как справляются с выбросами в A/B-тесте. Иногда просто наиболее крупных пользователей просто выкидывают из теста. В этой задаче мы посмотрим на 2 таких метода.


Для начала, давайте смоделируем искусственную задачу, на которой мы будем проверять критерии.

- Представим, что ваша метрика в тесте и в контроле взята из экспоненциального распределения (например - выручка). `scale` - любой.
- Пользователй в тесте и в контроле будет по 1000 человек.

Далее полученные тестовые и контрольные выборки будут обозначаться как `test`, `control`.

----
**1 задача:** проверить, что для таких выборок можно использовать t-test.

In [20]:
from statsmodels.stats.proportion import proportion_confint
from scipy.stats import expon, ttest_ind

In [36]:
size=1000
assert expon.mean(loc=10, scale=6) == expon.mean(loc=5, scale=11)

er = 0
for _ in range(10000):
    test = expon.rvs(loc=10, scale=6, size=size) 
    control = expon.rvs(loc=5, scale=11, size=size)
    
    pvalue = ttest_ind(control, test, alternative='two-sided').pvalue
    if pvalue < 0.05:
        er += 1

l_b, r_b = proportion_confint(count = er, nobs = 10000, alpha=0.05, method='wilson')

print(f'{er/10000}, [{l_b}, {r_b}]')

0.0504, [0.04628220197159746, 0.05486308936331493]


В целом, нормально :)

Теперь давайте перейдем к предлагаемым способам борьбы с выбросами. Для этого мы предлагаем следующие варианты: 

### Первый способ

Подобрать квантиль для отсечения топ пользователей в тесте и в контроле, причем в каждом случае подобран свой порог. 

То есть, новый критерий состоит из 2 этапов:    
1. Подобрать квантиль для отсечения в тесте и квантиль для отсечения в контроле.
    - Например: 
    ```
    outlier_control_filter = np.quantile(control, 0.99)
    outlier_test_filter = np.quantile(test, 0.99)
    
    control = control[control < outlier_control_filter]
    test    = test[test < outlier_test_filter]
    ```
2. Запустить на новых выборках t-test.

Проверьте, корректный ли в таком случае будет критерий? Поясните полученный результат.


In [51]:
size=1000
assert expon.mean(loc=10, scale=6) == expon.mean(loc=5, scale=11)

er = 0
for _ in range(10000):
    test = expon.rvs(loc=10, scale=6, size=size) 
    control = expon.rvs(loc=5, scale=11, size=size)
    
    # Отсекаем
    outlier_control_filter = np.quantile(control, 0.99)
    outlier_test_filter = np.quantile(test, 0.99)

    control = control[control < outlier_control_filter]
    test    = test[test < outlier_test_filter]
       
    # Считаем pvalue    
    pvalue = ttest_ind(control, test, alternative='two-sided').pvalue
    if pvalue < 0.05:
        er += 1

l_b, r_b = proportion_confint(count = er, nobs = 10000, alpha=0.05, method='wilson')

print(f'{er/10000}, [{l_b}, {r_b}]')

0.1301, [0.12364819512347855, 0.13683588687114798]


Очевидно, корректность критерий не проходит, так как рассчитанная ошибка очень отличается от заявленной.

По моему мнению, так происходит, потому что после отсечения выборки уже другие и можно сказать, что каждая выборка "поменяла" свое распределение, из-за этого тест Монте-Карло просто неправильно работает. У нас были выборки с разными дисперсиями, следовательно мы отсекли выборки с разными весами, если так можно сказать

### Второй способ:

1. Подобрать одну общую квантиль для отсечения в тесте и в контроле.
    - Например: 
    ```
    outlier_filter = np.quantile(np.concatenate([control, test]), 0.99)
    
    control = control[control < outlier_filter]
    test    = test[test < outlier_filter]
    ```
2. Запустить на новых выборках t-test.

#### 1 проверка

Проверьте, корректный ли в таком случае будет критерий? Поясните полученный результат.


In [52]:
size=1000
assert expon.mean(loc=10, scale=6) == expon.mean(loc=5, scale=11)

er = 0
for _ in range(10000):
    test = expon.rvs(loc=10, scale=6, size=size) 
    control = expon.rvs(loc=5, scale=11, size=size)
    
    # Отсекаем
    outlier_filter = np.quantile(np.concatenate([control, test]), 0.99)

    control = control[control < outlier_filter]
    test    = test[test < outlier_filter]
       
    # Считаем pvalue    
    pvalue = ttest_ind(control, test, alternative='two-sided').pvalue
    if pvalue < 0.05:
        er += 1

l_b, r_b = proportion_confint(count = er, nobs = 10000, alpha=0.05, method='wilson')

print(f'{er/10000}, [{l_b}, {r_b}]')

0.5979, [0.5882540451949481, 0.6074707679240734]


По хорошему, конечно, нельзя сказать, что критерий корректен, так как рассчитанная ошибка все еще больше заявленной, но это уже намного лучше. 

Здесь мы одинаково учитываем выбросы с двух выборок.

#### 2 проверка

Но и это еще не все. Теперь проверьте ваш критерий на симулированном A/B-тесте c эффектом, равным параметру `scale` в вашем распределении и проверьте гипотезу 

- $H_0: EA = EB$ + scale
- $H_1: EA \neq EB$ + scale

Для этого добавьте к каждому элементу в выборке test параметр scale: 
```
test = test + scale
```

1. Проверьте, что вы умеете с помощью ttest проверять заданную гипотезу. А точнее посмотрите, попадает ли в 95% процентах случаев в доверительный интервал истинный эффект `scale`?

2. Работает ли в таком случае предложенный критерий убирания выбросов?

3. Отличаются ли результаты у первой и у второй проверок? 
4. Сделайте вывод, можно ли использовать этот метод для убирания выбросов с точки зрения:
    - Теории.
    - Практики. Стали бы вы использовать в работе такой способ убирания выбросов?

In [55]:
size=1000
assert expon.mean(loc=10, scale=6) == expon.mean(loc=5, scale=11)

er = 0
for _ in range(10000):
    test = expon.rvs(loc=10, scale=2, size=size) 
    control = expon.rvs(loc=10, scale=2, size=size)
    
    # Добавляем scale
    test = test + 2
       
    # Считаем pvalue    
    pvalue = ttest_ind(control, test, alternative='two-sided').pvalue
    if pvalue < 0.05:
        er += 1

l_b, r_b = proportion_confint(count = er, nobs = 10000, alpha=0.05, method='wilson')

print(f'{er/10000}, [{l_b}, {r_b}]')

0.049, [0.044939516707863926, 0.053406849822486356]


In [42]:
expon.var(loc=5, scale=6) 

36.0

# Часть 2

## Задача 3 (2 балла)

### На зачет

Пусть нам нужно оценить $\mu$ — матожидание некой случайной величины, и у нас
есть выборка из нее размером 1000. С помощью бутстрапа соберем 1000 бутстрап-выборок, для каждой посчитаем среднее, получим 1000 средних. Это выборка из
1000 объектов, к ней можно применить критерий Стьюдента. Применим, построим
доверительный интервал для 𝜇.

Почему так делать нельзя?

Мы никак не построим доверительный интервал именно для $\mu$. Мы построим из выборки состоящей из оценок $\hat\mu$ только лишь доверительный интервал на среднее из этих оценок $\hat\mu$

In [120]:
from scipy.stats import gamma, sem
from tqdm.notebook import tqdm

In [121]:
N = 1000
example_dist = gamma(a=2, scale=3)
example_sample = example_dist.rvs(N)

theta = example_dist.mean()
theta_estim = np.mean(example_sample)

print(f"theta = {round(theta, 3)}")
print(f"theta estim = {round(theta_estim, 3)}")

theta = 6.0
theta estim = 6.089


In [122]:
def get_estim_theta_sample(sample_len, gen_sample_func, theta_func, B=1000):
    """
        Функция для генерации выборки theta estim.
    
        Праметры:
            - sample_len: какого размера выборку надо генерировать. 
                - sample_len = len(sample)
            - gen_sample_func: генерация выборки из нашего распределения, принимающая на вход лишь размер выборки. 
                - Например, gen_sample_func = lambda N: stats.norm().rvs(N)
            - theta_func: функция генерации оценки theta по выборке.
                - Например, lambda sample: numpy.percentile(sample, 75)
            - B: сколько выборок надо сгенерировать, какого размера будет массив theta estim
        Возвращает:
            - Массив theta_estim_array размера B 
    """
    
    theta_estim_array = []
    for _ in range(B):
        curr_sample = gen_sample_func(sample_len)
        theta_estim = theta_func(curr_sample)
        theta_estim_array.append(theta_estim)
    return theta_estim_array

In [125]:
win_theta = 0
win_theta_estim = 0
for _ in tqdm(range(1000)):

    gen_sample_func = lambda N: np.random.choice(example_sample, replace=True, size=N)

    theta_func = lambda sample: np.mean(sample)

    theta_estim_asterisk_array = get_estim_theta_sample(len(example_sample), gen_sample_func, theta_func, B=1000)
    l, r = t.interval(alpha=0.95, loc=np.mean(theta_estim_asterisk_array), df=len(theta_estim_asterisk_array)-1, scale=sem(theta_estim_asterisk_array)) 

    if l < theta < r:
        win_theta += 1
    if l < theta_estim < r:
        win_theta_estim += 1

print(f'Для оригинальной theta: {round(win_theta/1000, 4)*100}%')
print(f'Для оценки theta: {round(win_theta_estim/1000, 4)*100}%')

  0%|          | 0/1000 [00:00<?, ?it/s]

Для оригинальной theta: 0.0%
Для оценки theta: 95.39999999999999%


Что и требовалось доказать :)

## Задача 4 (3 балла)


Пусть $X_1,\ \cdots, X_n$ &mdash; некоторая выборка и $X_1^*,\ \cdots, X_n^*$ &mdash; построенная по ней бутстрап-выборка.

- С какой вероятностью элемент $X_i$ попадет в бутстрап-выборку?
- Посчитайте среднее число уникальных элементов в бутстрап-выборке, если в исходной выборке все элементы различны.

1) $$  \mathbb{P} = \frac{\sum_j[X_j = X_i]}{n}$$

2) $\mathbb{E} = \sum k*\mathbb{P}($количество уникальных$ = k)$

$\mathbb{P}($количество уникальных$ = k) = ? $

Способов выбрать k элементов из оригинальной выборки есть $C_n^k$.

Способов разместить выбранные k элементов по бутстрап-выборке есть $A_n^k = \frac{n!}{(n-k)!}$

Вероятность реализации одного такого способа: $(\frac{1}{n})^n$

Таким образом, $$ \mathbb{E} = \sum k* \frac{n!}{(n-k)!k!} * \frac{n!}{(n-k)!} * \frac{1}{n^n} $$

## Задача 5 (4 балла)

### На зачет

Пусть у вас выборка из неизвестного распределения (но все элементы больше 0) и вы хотите проверить гипотезу:
$$
\begin{align}
&H_0: E \left[\dfrac{median(X)}{\overline{X}} \right] = \theta_0\ vs.\\
&H_1: E \left[\dfrac{median(X)}{\overline{X}} \right] \neq \theta_0
\end{align}
$$
* черточка означает среднее по выборке.

С помощью бутстрапа постройте критерий уровня значимости $\alpha$ для проверки этой гипотезы.

Для этого вам надо:
- Показать, в каких случаях вы отвергаете $H_0$
- Как посчитать p-value.

Для начала теоретически поясните ваш алгоритм действий.

Формируем бутстрап-выборки и считаем для них `theta_func = = lambda sample: np.median(sample, axis=1)/np.mean(sample, axis=1)`

Далее получаем выборку из оценок theta. Берем квантили: 

`left_theta_asterisk, right_theta_asterisk = np.quantile(theta_asterisk_array, [alpha/2, 1-alpha/2])`

И если $\theta_0 \in $`[left_theta_asterisk, right_theta_asterisk]`, то нулевая гипотеза не отвергается.

Далее я увидел, что еще нужно pvalue посчитать. Я выбрал такой путь: нам как-то надо получить распределение theta, чтобы сказать кто более экстремальный относительно полученной theta. У нас есть выборка, можно воспользоваться эмпирическим распределением. Итак, $$F_{emp}(\theta_0) = \frac{1}{n} \cdot \sum [\theta_i < \theta_0]$$

Таким образом, $pvalue = 1- F_{emp}(\theta_0)$. (Там еще некоторые модификации для двустороннего случая, они учтены в коде)

В конце критерий проверен Монте-Карлом

Теперь перйдем к практике:

Какими библиотеками вы можете пользоваться:
```
import numpy as np
from collections import namedtuple
from scipy.stats.binom
from scipy.stats.norm
from scipy.stats.t
from scipy.stats.bernoulli
```

In [2]:
import numpy as np
from collections import namedtuple
from scipy.stats import binom
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import bernoulli

**Чтобы код прошел автопроверку, используйте процентильный доверительный интервал!**

In [22]:
#Автопроверка

MyStrangeStatResults = namedtuple('MyStrangeStatResults', 
                                  ['is_rejected', 'pvalue'])


def bootstrap_strange_stat_checker(sample: list, mu_0: float, alpha: float = 0.05):
    """
    Параметры:
    - sample: текущая реализация выборки
    - mu_0: значение странной синусной статистики при H_0
    - alpha: уровень значимости критерия.
        
    Возвращает:
    - MyStrangeStatResults с полями:
        - is_rejected: bool
            - отверглась или нет гипотеза H_0 на уровне значимости alpha
        - pvalue: float
    """
    boot_samples_size = 1000
    batch_size = boot_samples_size // 20
    N = len(sample)
    is_rejected = None
    pvalue = None
    theta_func = lambda sample: np.median(sample, axis=1)/np.mean(sample, axis=1)

    #<Ваш код>
    #Используйте процентильный доверительный интервал!
    theta_estim = theta_func(np.expand_dims(sample, axis=0)).ravel()
    assert len(theta_estim) == 1
    theta_estim = theta_estim[0]
    
    theta_asterisk_array = []
    for _ in range(0, boot_samples_size, batch_size):
        bootstrap_sample = np.random.choice(sample, replace=True, size=(batch_size, N))
        theta_asterisk = theta_func(bootstrap_sample).ravel()
        assert len(theta_asterisk) == batch_size
        theta_asterisk_array = np.concatenate([theta_asterisk_array, theta_asterisk])
    left_theta_asterisk, right_theta_asterisk = np.quantile(theta_asterisk_array, [alpha/2, 1-alpha/2])
    
    F_mu0 = sum(theta_asterisk_array < mu_0)/len(theta_asterisk_array)
    pvalue = 1 - F_mu0
    if pvalue > 0.5:
        pvalue = F_mu0
    pvalue = pvalue * 2
    
    if left_theta_asterisk < mu_0 < right_theta_asterisk:
        is_rejected = False
    else:
        is_rejected = True
    

    return MyStrangeStatResults(is_rejected, pvalue)

Теперь проверим на время ваш код

In [4]:
import scipy.stats as sps
from tqdm.notebook import tqdm

In [23]:
# Этот код на сервере академии должен выполняться не более 2 минут,
# иначе он не пройдет проверку по времени
for i in tqdm(range(1000)):
    sample = sps.expon(loc=1, scale=1000).rvs(1000)
    bootstrap_strange_stat_checker(sample, mu_0=10, alpha=0.05)
        

  0%|          | 0/1000 [00:00<?, ?it/s]

Проверим Монте_карлом

In [24]:
bad_count = 0
for i in tqdm(range(1000)):
    sample = sps.expon(loc=1, scale=1000).rvs(1000)
    mu_0 = sps.expon(loc=1, scale=1000).median()/sps.expon(loc=1, scale=1000).mean()
    if bootstrap_strange_stat_checker(sample, mu_0=mu_0, alpha=0.05).is_rejected:
        bad_count += 1
l_b, r_b = proportion_confint(count = bad_count, nobs = 1000, alpha=0.05, method='wilson')
print(f'{bad_count/1000}, CI = [{l_b}, {r_b}]')

  0%|          | 0/1000 [00:00<?, ?it/s]

0.042, CI = [0.031220885546165443, 0.05628442522660306]


Отлично, критерий корректен

## Задача 6 (2 балла)

Эту задачу стоит решать только после второй задачи из домашки.

Пусть мы подозреваем, что в AB-тесте в тестовой группе увеличилась дисперсия по сравнению с контрольной группой. Сформулировать это предположение на языке статистики и
реализовать его проверку с помощью бутстрапа.

**Ваши теоретические выкладки**


Сформулируем гипотезы. Наша оценка theta в данном случае есть выборочная дисперсия, и мы хотим проверить смещена ли оценка на тесте относительно контроля.

$ H_0: \mathbb{E}_{test}[\frac{1}{n-1}\sum(X_i - \overline{X})^2] = \mathbb{E}_{control}[\frac{1}{n-1}\sum(X_i - \overline{X})^2]$

$ H_1: \mathbb{E}_{test}[\frac{1}{n-1}\sum(X_i - \overline{X})^2] > \mathbb{E}_{control}[\frac{1}{n-1}\sum(X_i - \overline{X})^2] $

Формируем две бутстрап-выборки, считаем для них выборочные дисперсии. Получим 2 выборки выборочных дисперсий для теста и контроля, далее считаем разницу этих выборок, берем перцентильный доверительный интервал (только односторонний, так как наша гипотеза односторонняя по условию) и есть ли там 0, то не нулевая гипотеза не отвергается. 

pvalue я считаю так же как и в прошлой задаче, через эмпирическое распределение.

Теперь перйдем к практике:

Какими библиотеками вы можете пользоваться:
```
import numpy as np
from collections import namedtuple
from scipy.stats.binom
from scipy.stats.norm
from scipy.stats.t
from scipy.stats.bernoulli
```

In [26]:
import numpy as np
from collections import namedtuple
from scipy.stats import binom
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import bernoulli

In [33]:
#Автопроверка

CompareVarianceResults = namedtuple('CompareVarianceResults', 
                                  ['is_rejected', 'pvalue'])


def variance_ab_checker(control: list, test: list, alpha: float = 0.05):
    """
    Параметры:
    - control: контрольная группа
    - test: тестовая группа
    - alpha: уровень значимости критерия.
        
    Возвращает:
    - CompareVarianceResults с полями:
        - is_rejected: bool
            - отверглась или нет гипотеза H_0 на уровне значимости alpha
        - pvalue: float
    """
    boot_samples_size = 1000
    batch_size = boot_samples_size // 20
    N_test = len(test)
    N_control = len(control)
    is_rejected = None
    pvalue = None
    left_bound = None
    right_bound = None
    theta_func = lambda sample: np.var(sample, ddof=1, axis=1)

    theta_array = []
    for _ in range(0, boot_samples_size, batch_size):
        # Генерируем сразу batch_size выборок
        bootstrap_test = np.random.choice(test, replace=True, size=(batch_size, N_test))
        bootstrap_control = np.random.choice(control, replace=True, size=(batch_size, N_control))
            
        theta_test = theta_func(bootstrap_test).ravel()
        theta_control = theta_func(bootstrap_control).ravel()
        assert len(theta_test) == batch_size and len(theta_control) == batch_size
        theta_array = np.concatenate([theta_array, theta_test - theta_control])
    left_theta = np.quantile(theta_array, alpha)
    if left_theta > 0:
        is_rejected = True
    else:
        is_rejected = False
    
    pvalue = sum(theta_array < 0)/len(theta_array)


    return CompareVarianceResults(is_rejected, pvalue)

In [34]:
import scipy.stats as sps
from tqdm.notebook import tqdm

In [ ]:
# Этот код на сервере академии должен выполняться не более 10 минут,
# иначе он не пройдет проверку по времени
for i in tqdm(range(1000)):
    test = sps.expon(loc=1, scale=1000).rvs(1000)
    control = sps.expon(loc=1, scale=1000).rvs(1000)
    variance_ab_checker(test, control, alpha=0.05)

In [37]:
bad_count = 0
for i in tqdm(range(1000)):
    test = sps.expon(loc=1, scale=1000).rvs(1000)
    control = sps.expon(loc=1, scale=1000).rvs(1000)
    if variance_ab_checker(test, control, alpha=0.05).is_rejected:
        bad_count += 1
l_b, r_b = proportion_confint(count = bad_count, nobs = 1000, alpha=0.05, method='wilson')
print(f'{bad_count/1000}, CI = [{l_b}, {r_b}]')

  0%|          | 0/1000 [00:00<?, ?it/s]

0.059, CI = [0.04601429821280047, 0.07536090277145915]
